In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import os
import pandas as pd

# Path to the folder containing all JSON files
file_path = '/content/drive/My Drive/restaurant_data/rde2f7e.json'
df = pd.read_json(file_path)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
import pandas as pd

# Extract relevant columns
features = df[['blur_restaurant_uuid', 'blur_customer_order', 'invoice_unix_timestamp']]

# Extract 'blur_name' and 'blur_quantity' from 'blur_customer_order'
features['blur_name'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_name'] if isinstance(x, list) and len(x) > 0 and 'blur_name' in x[0] else None)
features['blur_quantity'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_quantity'] if isinstance(x, list) and len(x) > 0 and 'blur_quantity' in x[0] else None)

# Convert 'invoice_unix_timestamp' to datetime format and extract year, month, day
features['invoice_date'] = pd.to_datetime(features['invoice_unix_timestamp'], unit='s').dt.strftime('%Y-%m-%d')

grouped_features = features.groupby(['blur_name', 'invoice_date'], as_index=False).agg({'blur_quantity': 'sum'})
# Sort values within each group based on 'invoice_date'in ascending order, 'blur_quantity' in descending order
grouped_features.sort_values(['invoice_date', 'blur_quantity'], ascending=[True, False], inplace=True)

<ipython-input-4-99624f76f09f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['blur_name'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_name'] if isinstance(x, list) and len(x) > 0 and 'blur_name' in x[0] else None)
<ipython-input-4-99624f76f09f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['blur_quantity'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_quantity'] if isinstance(x, list) and len(x) > 0 and 'blur_quantity' in x[0] else None)
<ipython-input-4-99624f76f

In [ ]:
import pandas as pd

# Assuming your DataFrame is named grouped_features
# If not, replace grouped_features with the actual name of your DataFrame

# Create a new column 'ranking' based on the order of 'blur_quantity' for each 'invoice_date' group
grouped_features['ranking'] = grouped_features.groupby('invoice_date')['blur_quantity'].rank(ascending=False, method='min').astype(int)
print(grouped_features)

            blur_name invoice_date  blur_quantity  ranking
18868        [5040]奶茶   2020-06-12             11        1
3339       [17a4]紅茶拿鐵   2020-06-12              7        2
1            [108a]薯餅   2020-06-12              4        3
54872      [b8e2]豬排吐司   2020-06-12              4        3
55745        [b947]紅茶   2020-06-12              4        3
...               ...          ...            ...      ...
66472  [e468]咔拉雞吐司 原味   2023-10-06              1       25
69109      [f0b5]鮮蝦漢堡   2023-10-06              1       25
69168       [f458]*號餐   2023-10-06              1       25
69971      [f54d]拿鐵咖啡   2023-10-06              1       25
72528     [fc36]*元三明治   2023-10-06              1       25

[73883 rows x 4 columns]


# Train the model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


# Features: 'invoice_date', 'blur_name'
X = grouped_features[['invoice_date', 'blur_name']].copy()

# Convert 'invoice_date' to numerical representation
X['invoice_date'] = pd.to_datetime(X['invoice_date']).astype(int)

# Encode 'blur_name' using one-hot encoding or other suitable encoding methods
# Example using pandas get_dummies for one-hot encoding
X = pd.get_dummies(X, columns=['blur_name'], drop_first=True)

# Target variable
y = grouped_features['ranking'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestRegressor
#increase
n_estimators = 100
max_depth = 100
#smaller
min_samples_split = 20
#bigger
min_samples_leaf = 10
max_features = None

random_forest_model = RandomForestRegressor(
    random_state=42,
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_features=max_features
)

# Train the model
random_forest_model.fit(X_train, y_train)

# Predictions
y_pred_rf = random_forest_model.predict(X_test)

# Evaluate the model using R-squared
r_squared_rf = r2_score(y_test, y_pred_rf)
print(f'R-squared (Random Forest): {r_squared_rf:.2f}')



R-squared (Random Forest): 0.53


##prediction

In [ ]:
import pandas as pd

file_path1 = '/content/drive/My Drive/restaurant_data/r5028cc.json'
df = pd.read_json(file_path1)
# print (df)

In [ ]:
import pandas as pd

# Extract relevant columns
features = df[['blur_restaurant_uuid', 'blur_customer_order', 'invoice_unix_timestamp']]

# Extract 'blur_name' and 'blur_quantity' from 'blur_customer_order'
features['blur_name'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_name'] if isinstance(x, list) and len(x) > 0 and 'blur_name' in x[0] else None)
features['blur_quantity'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_quantity'] if isinstance(x, list) and len(x) > 0 and 'blur_quantity' in x[0] else None)

# Convert 'invoice_unix_timestamp' to datetime format and extract year, month, day
features['invoice_date'] = pd.to_datetime(features['invoice_unix_timestamp'], unit='s').dt.strftime('%Y-%m-%d')

grouped_features = features.groupby(['blur_name', 'invoice_date'], as_index=False).agg({'blur_quantity': 'sum'})
# Sort values within each group based on 'invoice_date'in ascending order, 'blur_quantity' in descending order
grouped_features.sort_values(['invoice_date', 'blur_quantity'], ascending=[True, False], inplace=True)
unique_blur_names = grouped_features['blur_name'].nunique()

print(grouped_features)






print(unique_blur_names)







<ipython-input-384-4acff17c0400>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['blur_name'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_name'] if isinstance(x, list) and len(x) > 0 and 'blur_name' in x[0] else None)
<ipython-input-384-4acff17c0400>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['blur_quantity'] = df['blur_customer_order'].apply(lambda x: x[0]['blur_quantity'] if isinstance(x, list) and len(x) > 0 and 'blur_quantity' in x[0] else None)


            blur_name invoice_date  blur_quantity
21000     [5519]火腿蛋吐司   2020-07-07              4
65696      [de9a]紅茶 中   2020-07-07              4
22059      [5ae2]電話訂單   2020-07-07              3
29507      [76b5]紅茶 大   2020-07-07              3
46441      [adc5]全麥蛋餅   2020-07-07              3
...               ...          ...            ...
61358  [d3b4]卡啦雞腿丹麥吐司   2023-10-06              1
62818    [d7d2]薯泥起司蛋餅   2023-10-06              1
67945       [e1b5]小肉豆   2023-10-06              1
72688      [f8bf]熱狗蛋餅   2023-10-06              1
74680      [ffa8]可可 大   2023-10-06              1

[74681 rows x 3 columns]
488


<ipython-input-384-4acff17c0400>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['invoice_date'] = pd.to_datetime(features['invoice_unix_timestamp'], unit='s').dt.strftime('%Y-%m-%d')


In [ ]:
import pandas as pd
import numpy as np

# Define the date range
prediction_dates = pd.date_range(start='2023-10-07', end='2023-10-13')

# Create a list of unique blur_names from your original data
unique_blur_names = grouped_features['blur_name'].unique()

# Create a DataFrame with all combinations of dates and blur_names
prediction_data = pd.DataFrame(index=pd.MultiIndex.from_product([prediction_dates, unique_blur_names], names=['invoice_date', 'blur_name'])).reset_index()

# Convert 'invoice_date' to numerical representation
prediction_data['invoice_date'] = pd.to_datetime(prediction_data['invoice_date']).astype(int)

# Encode 'blur_name' using one-hot encoding or other suitable encoding methods
# Example using pandas get_dummies for one-hot encoding
prediction_data = pd.get_dummies(prediction_data, columns=['blur_name'], drop_first=True)

print(prediction_data)


             invoice_date  blur_name_[1060]鮪魚總匯  blur_name_[10c2]卡啦雞總匯  \
0     1696636800000000000                     0                      0   
1     1696636800000000000                     0                      0   
2     1696636800000000000                     0                      0   
3     1696636800000000000                     0                      0   
4     1696636800000000000                     0                      0   
...                   ...                   ...                    ...   
3411  1697155200000000000                     0                      0   
3412  1697155200000000000                     0                      0   
3413  1697155200000000000                     0                      0   
3414  1697155200000000000                     0                      0   
3415  1697155200000000000                     0                      0   

      blur_name_[10c9]草莓  blur_name_[1164]法式套餐  blur_name_[1223]黑胡椒豬排  \
0                      0              

##predict results

In [ ]:
# Make predictions
predictions = random_forest_model.predict(prediction_data)

# Add the predictions to the prediction_data DataFrame
prediction_data['predicted_ranking'] = predictions

# print(prediction_data)

In [ ]:

# Extract 'invoice_date' column from prediction_data
date_column = prediction_data['invoice_date']

# Convert the numerical date to datetime format
date_column = pd.to_datetime(date_column)

# Create a new DataFrame with the transformed 'invoice_date'
result_df = pd.DataFrame({'invoice_date': date_column})






# Extract relevant column names from prediction_data
# type the former value-1
blur_name_columns = prediction_data.columns[1:487]







# Create a new column in invoice_date_df with the corresponding blur_name
result_df['blur_name'] = prediction_data[blur_name_columns].idxmax(axis=1).str.replace('blur_name_', '')

# Print the new DataFrame
print(result_df)


     invoice_date         blur_name
0      2023-10-07       [5519]火腿蛋吐司
1      2023-10-07        [de9a]紅茶 中
2      2023-10-07        [5ae2]電話訂單
3      2023-10-07        [76b5]紅茶 大
4      2023-10-07        [adc5]全麥蛋餅
...           ...               ...
3411   2023-10-13      [2795]紐奧良雞腿堡
3412   2023-10-13  [b739]紐澳良雞腿嫩蛋米漢堡
3413   2023-10-13    [bec3]胡麻里肌丹麥吐司
3414   2023-10-13   [fc13]泡菜豬排嫩蛋米漢堡
3415   2023-10-13     [fe06]墨西哥烤 *隻

[3416 rows x 2 columns]


In [ ]:

# print(prediction_data['predicted_ranking'])
import numpy as np






# Assuming 163 rows per group
#type the former value
group_size = 488




# Initialize an empty list to store the final rankings
final_rankings = []

# Iterate through each group
for group_start in range(0, len(prediction_data), group_size):
    # Select the current group
    group = prediction_data['predicted_ranking'].iloc[group_start:group_start + group_size]

    # Calculate the ranking within the group
    group_ranking = np.argsort(group)
    group_ranking = np.argsort(group_ranking)+1

    # Extend the final_rankings list with the rankings for the current group
    final_rankings.extend(group_ranking)

# Add the 'final_ranking' column to the DataFrame
prediction_data['final_ranking'] = final_rankings

# Print the DataFrame with the new 'final_ranking' column
# print(prediction_data[['predicted_ranking', 'final_ranking']])



In [ ]:
# Merge 'final_ranking' column from prediction_data into result_df
result_df = pd.merge(result_df, prediction_data[['final_ranking']], left_index=True, right_index=True, how='left')

# Print the updated result_df
# print(result_df)

# Extract restaurant name from the file_path
restaurant_name = os.path.basename(file_path).split('.')[0]
result_df['restaurant_name'] = restaurant_name

result_df.to_csv('/content/drive/My Drive/restaurant_data/decisiontree_predicted_r5028cc.csv', index=False)
print(result_df)

     invoice_date         blur_name  final_ranking restaurant_name
0      2023-10-07       [5519]火腿蛋吐司              7         r5028cc
1      2023-10-07        [de9a]紅茶 中              3         r5028cc
2      2023-10-07        [5ae2]電話訂單              1         r5028cc
3      2023-10-07        [76b5]紅茶 大              2         r5028cc
4      2023-10-07        [adc5]全麥蛋餅            331         r5028cc
...           ...               ...            ...             ...
3411   2023-10-13      [2795]紐奧良雞腿堡            160         r5028cc
3412   2023-10-13  [b739]紐澳良雞腿嫩蛋米漢堡            159         r5028cc
3413   2023-10-13    [bec3]胡麻里肌丹麥吐司            158         r5028cc
3414   2023-10-13   [fc13]泡菜豬排嫩蛋米漢堡            394         r5028cc
3415   2023-10-13     [fe06]墨西哥烤 *隻            478         r5028cc

[3416 rows x 4 columns]
